In [1]:
from datetime import datetime

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import os
import json
import pandas as pd
import numpy as np

import matplotlib as mlp
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname = "C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family = font_name)
mlp.rcParams["axes.unicode_minus"] = False

### 7 classes

- 0 정상
<br><br>
- 1 곰팡 초기
- 2 곰팡 중기
- 3 곰팡 말기
<br><br>
- 4 말림 초기
- 5 말림 중기
- 6 말림 말기

In [79]:
yolo_df1 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_train(leaf).csv")
yolo_df2 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_질병_train.csv")
yolo_df3 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_증강_train.csv")

yolo_df4 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_정상_val(leaf).csv")
yolo_df5 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_질병_val.csv")
yolo_df6 = pd.read_csv("data/tomato_label/tomato_set/[라벨]토마토_증강_val.csv")

In [80]:
yolo_df = pd.concat([yolo_df1, yolo_df2, yolo_df3, yolo_df4, yolo_df5, yolo_df6])

In [81]:
# yolo_df.to_csv("data/tomato_label/tomato_set/[라벨]통합.csv", encoding="utf-8-sig", index=False)

In [82]:
yolo_df

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,1481,1345,2719,3233
1,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01_...,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,1480,1365,2710,3230
2,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,3095,930,4480,2980
3,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01_...,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,3105,940,4475,2985
4,V006_77_0_00_11_03_12_0_c01_20201013_0002_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,2130,205,4015,2655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0005...,2020/11/30,4032,3024,77,1,17.0,19,11,3,12,3,608,698,3487,2686
2796,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0006...,2020/11/30,3024,4032,77,1,17.0,19,11,3,12,3,698,545,2686,3423
2797,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0007...,2020/11/30,3024,4032,77,1,17.0,19,11,3,12,3,698,545,2686,3423
2798,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0008...,2020/11/30,4032,3024,77,1,17.0,19,11,3,12,3,545,698,3423,2686


In [83]:
yolo_df.risk.value_counts()

2    13943
1     6516
0     6360
3     3561
Name: risk, dtype: int64

In [84]:
yolo_df[["disease", "risk"]].value_counts()

disease  risk
19       2       8608
0        0       6360
18       2       5335
         1       3817
19       1       2699
         3       2197
18       3       1364
dtype: int64

In [85]:
yolo_df[(yolo_df["disease"]==18) & (yolo_df["risk"]==1)]

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr
0,V006_77_1_18_11_03_13_1_3248b_20201005_57.jpg,2020/10/05,4032,3024,77,1,4.0,18,11,3,13,1,908,916,3104,2312
1,V006_77_1_18_11_03_13_1_3248b_20201012_60.jpg,2020/10/12,4032,3024,77,1,4.0,18,11,3,13,1,1108,556,3356,2260
2,V006_77_1_18_11_03_13_1_3248b_20201012_61.jpg,2020/10/12,4032,3024,77,1,4.0,18,11,3,13,1,1152,772,3232,2420
3,V006_77_1_18_11_03_13_1_3248b_20201012_64.jpg,2020/10/12,4032,3024,77,1,4.0,18,11,3,13,1,540,544,2756,2224
4,V006_77_1_18_11_03_13_1_3248b_20201019_67.jpg,2020/10/19,4032,3024,77,1,4.0,18,11,3,13,1,648,852,3340,2524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,V006_77_1_18_11_03_13_1_9720w_20201109_192_a00...,2020/11/09,4032,3024,77,1,4.0,18,11,3,13,1,1344,858,3270,2320
476,V006_77_1_18_11_03_13_1_9720w_20201109_192_a00...,2020/11/09,3024,4032,77,1,4.0,18,11,3,13,1,858,762,2320,2688
477,V006_77_1_18_11_03_13_1_9720w_20201109_192_a00...,2020/11/09,3024,4032,77,1,4.0,18,11,3,13,1,858,762,2320,2688
478,V006_77_1_18_11_03_13_1_9720w_20201109_192_a00...,2020/11/09,4032,3024,77,1,4.0,18,11,3,13,1,1344,858,3270,2320


In [86]:
yolo_df["class_no"] = None

In [87]:
yolo_df

,image,date,width,height,task,type,region,disease,crop,area,grow,risk,xtl,ytl,xbr,ybr,class_no
0,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,1481,1345,2719,3233,None
1,V006_77_0_00_11_03_12_0_c01_20201013_0000_S01_...,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,1480,1365,2710,3230,None
2,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,3095,930,4480,2980,None
3,V006_77_0_00_11_03_12_0_c01_20201013_0001_S01_...,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,3105,940,4475,2985,None
4,V006_77_0_00_11_03_12_0_c01_20201013_0002_S01.JPG,2020/10/13,6000,4000,77,0,NaN,0,11,3,12,0,2130,205,4015,2655,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0005...,2020/11/30,4032,3024,77,1,17.0,19,11,3,12,3,608,698,3487,2686,None
2796,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0006...,2020/11/30,3024,4032,77,1,17.0,19,11,3,12,3,698,545,2686,3423,None
2797,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0007...,2020/11/30,3024,4032,77,1,17.0,19,11,3,12,3,698,545,2686,3423,None
2798,V006_77_1_19_11_03_12_3_9357q_20201130_9_a0008...,2020/11/30,4032,3024,77,1,17.0,19,11,3,12,3,545,698,3423,2686,None


In [126]:
yolo_df.loc[(yolo_df.disease==0) & (yolo_df.risk==0), "class_no"] = 0

yolo_df.loc[(yolo_df.disease==18) & (yolo_df.risk==1), "class_no"] = 1
yolo_df.loc[(yolo_df.disease==18) & (yolo_df.risk==2), "class_no"] = 2
yolo_df.loc[(yolo_df.disease==18) & (yolo_df.risk==3), "class_no"] = 3

yolo_df.loc[(yolo_df.disease==19) & (yolo_df.risk==1), "class_no"] = 4
yolo_df.loc[(yolo_df.disease==19) & (yolo_df.risk==2), "class_no"] = 5
yolo_df.loc[(yolo_df.disease==19) & (yolo_df.risk==3), "class_no"] = 6

In [149]:
yolo_df.class_no.value_counts().sort_index()

0    6360
1    3817
2    5335
3    1364
4    2699
5    8608
6    2197
Name: class_no, dtype: int64

In [167]:
for i in range(0, 7):
    print((yolo_df.class_no.value_counts().sort_index()[i]/30380 * 100).round(1))

20.9
12.6
17.6
4.5
8.9
28.3
7.2


#### class 불균형

In [184]:
# yolo_df.to_csv("data/tomato_label/tomato_set/[라벨]통합_class_no.csv", encoding="utf-8-sig", index=False)

In [201]:
yolo_data = pd.read_csv("data/tomato_label/tomato_set/[라벨]통합_class_no.csv")

In [202]:
yolo_df = yolo_data.copy()

In [203]:
## YOLO 기준 컬럼 추가
### cx, cy, w, h

yolo_df["cx"] = (yolo_df["xtl"] + yolo_df["xbr"])/2
yolo_df["cy"] = (yolo_df["ytl"] + yolo_df["ybr"])/2
yolo_df["w"] = yolo_df["xbr"] - yolo_df["xtl"]
yolo_df["h"] = yolo_df["ybr"] - yolo_df["ytl"]

## BB 좌표 비정상 조건 확인
## 0이 있으면 안되는 경우

print("xbr=0 인 경우")
print(yolo_df[yolo_df.xbr == 0])
print("ybr=0 인 경우")
print(yolo_df[yolo_df.ybr == 0])


##  조건 확인
### xbr > xtl
print("xbr > xtl 아닌 경우")
print(yolo_df[yolo_df.xbr < yolo_df.xtl])
### ybr > ytl 
print("ybr > ytl 아닌 경우")
print(yolo_df[yolo_df.ybr < yolo_df.ytl])

## cx, cy, w, h 이미지 크기 기준, 비율로 정규화
### 0~1
yolo_df["cx"] = yolo_df["cx"]/yolo_df["width"]
yolo_df["cy"] = yolo_df["cy"]/yolo_df["height"]
yolo_df["w"] = yolo_df["w"]/yolo_df["width"]
yolo_df["h"] = yolo_df["h"]/yolo_df["height"]

## cx cy w h, 0~1 사이 값인지 확인

print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["cx"] >= 0) & (yolo_df["cx"] <= 1)].cx))
print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["cy"] >= 0) & (yolo_df["cy"] <= 1)].cy))
print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["w"] >= 0) & (yolo_df["w"] <= 1)].w))
print("(df row수) - (0~1 사이인 row수)/n =>", len(yolo_df) - len(yolo_df[(yolo_df["h"] >= 0) & (yolo_df["h"] <= 1)].h))


## 0~1 아닌 것 조회
# 0보다 작은 경우가... 설마 있을까?
print("0보다 작은 경우")
print(yolo_df[(yolo_df["cx"] < 0)].cx)
print(yolo_df[(yolo_df["cy"] < 0)].cy)
print(yolo_df[(yolo_df["w"] < 0)].w)
print(yolo_df[(yolo_df["h"] < 0)].h)

print("1보다 큰 경우")
print(yolo_df[(yolo_df["cx"] > 1)])
print(yolo_df[(yolo_df["cy"] > 1)])
print(yolo_df[(yolo_df["w"] > 1)])
print(yolo_df[(yolo_df["h"] > 1)])

########################################################################

## 컬럼 정리, copy
yolo_df.head(0)
yolo_data = yolo_df.copy()
#
yolo_data = yolo_data.reindex(columns=["class_no", "cx", "cy", "w", "h"])
yolo_data.head(0)

## 타입 => 문자열
yolo_data = yolo_data.astype("str")

xbr=0 인 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, class_no, cx, cy, w, h]
Index: []

[0 rows x 21 columns]
ybr=0 인 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, class_no, cx, cy, w, h]
Index: []

[0 rows x 21 columns]
xbr > xtl 아닌 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, class_no, cx, cy, w, h]
Index: []

[0 rows x 21 columns]
ybr > ytl 아닌 경우
Empty DataFrame
Columns: [image, date, width, height, task, type, region, disease, crop, area, grow, risk, xtl, ytl, xbr, ybr, class_no, cx, cy, w, h]
Index: []

[0 rows x 21 columns]
(df row수) - (0~1 사이인 row수)/n => 0
(df row수) - (0~1 사이인 row수)/n => 0
(df row수) - (0~1 사이인 row수)/n => 0
(df row수) - (0~1 사이인 row수)/n => 0
0보다 작은 경우
Series([], Name: cx, dtype: float64)
Series([], Name: cy, dtype:

In [204]:
yolo_df.image

0        V006_77_0_00_11_03_12_0_c01_20201013_0000_S01.JPG
1        V006_77_0_00_11_03_12_0_c01_20201013_0000_S01_...
2        V006_77_0_00_11_03_12_0_c01_20201013_0001_S01.JPG
3        V006_77_0_00_11_03_12_0_c01_20201013_0001_S01_...
4        V006_77_0_00_11_03_12_0_c01_20201013_0002_S01.JPG
                               ...                        
30375    V006_77_1_19_11_03_12_3_9357q_20201130_9_a0005...
30376    V006_77_1_19_11_03_12_3_9357q_20201130_9_a0006...
30377    V006_77_1_19_11_03_12_3_9357q_20201130_9_a0007...
30378    V006_77_1_19_11_03_12_3_9357q_20201130_9_a0008...
30379    V006_77_1_19_11_03_12_3_9357q_20201130_9_a0009...
Name: image, Length: 30380, dtype: object

In [205]:
# 저장 경로 확인

# num_rows = len(yolo_data)
num_rows = len(yolo_df)

for i in tqdm(range(num_rows)):
# for i in range(4):
    value = " ".join(yolo_data.iloc[i])

    # 파일 저장 위치!!
    filename = f"data/tomato_yolo_shuffle_2/train/labels/{yolo_df['image'][i].split('.')[0]}.txt"
    
    with open(filename, "w") as f:
        f.write(value)
        

num_files = len(os.listdir("data/tomato_yolo_shuffle_2/train/labels"))
print(f"경로 내 총 파일 수 : {num_files}")

100%|██████████████████████████████████████████████████████████████████████████| 30380/30380 [00:21<00:00, 1414.94it/s]

경로 내 총 파일 수 : 30380


## 사진 이동 / shuffle

In [156]:
move_df = pd.read_csv("data/tomato_label/tomato_set/move_list_names.csv")
leaf_df = pd.read_csv("data/tomato_label/tomato_set/no_leaf_names.csv")

In [ ]:
C:\final_project\tomato_yolo_set

In [162]:
move_list = list(move_df.image)

In [163]:
leaf_list = list(leaf_df.image)

In [164]:
move_out_list = move_list + leaf_list

In [206]:
import os
import shutil

In [217]:
label_list = os.listdir("data/tomato_yolo_shuffle_2/train/labels")[0].split(".")[0]

In [221]:
val_list = os.listdir("data/tomato_yolo_shuffle_2/val/images")[0].split(".")[0]
os.listdir("data/tomato_yolo_shuffle_2/val/images")

In [224]:
os.listdir("data/tomato_yolo_shuffle_2/val/images")

['V006_77_0_00_11_03_12_0_c01_20201013_0005_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c01_20201013_0009_S01.JPG',
 'V006_77_0_00_11_03_12_0_c01_20201013_0011_S01.JPG',
 'V006_77_0_00_11_03_12_0_c01_20201013_0012_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c01_20201013_0023_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c01_20201013_0025_S01.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0003_S01.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0005_S01.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0007_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0012_S01.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0014_S01.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0030_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0031_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0032_S01.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0033_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0045_S01_1.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0059_S01.JPG',
 'V006_77_0_00_11_03_12_0_c02_20201013_0062_S01_1.JPG',
 'V006_77_

In [223]:
len(os.listdir("data/tomato_yolo_shuffle_2/val/images"))

3038

In [ ]:
os.listdir("data/tomato_yolo_shuffle_2/val/images")

In [226]:
val_new = []
for i in range(len(os.listdir("data/tomato_yolo_shuffle_2/train/labels"))):
    if 
    val_new.append(os.listdir("data/tomato_yolo_shuffle_2/train/labels")[i].split(".")[0])
    os.listdir("data/tomato_yolo_shuffle_2/val/images")

In [ ]:
val_new

In [225]:
import os
import shutil

# 이동시키고자 하는 파일들의 리스트

# val
file_list = os.listdir("data/tomato_yolo_shuffle_2/val/images")

# test
# file_list = 


# 원본 폴더 경로
# source_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/train/images"
# source_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/val/images"

source_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo_shuffle_2/train/labels"

# 대상 폴더 경로
# target_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo/move_out/train_out"

target_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo_shuffle_2/val/labels"
# target_folder = "C:/juna27/PythonWork/AI/data/tomato_yolo_shuffle_2/test/labels"


for file_name in file_list:
    # 파일의 전체 경로 생성
    source_file_path = os.path.join(source_folder, file_name)
    
    # 파일 존재 여부 확인
    if os.path.exists(source_file_path):
        # 대상 폴더에 파일 이동
        shutil.move(source_file_path, target_folder)
        print(f'{file_name}을(를) 이동했습니다.')
    else:
        print(f'{file_name}을(를) 찾을 수 없습니다.')

V006_77_0_00_11_03_12_0_c01_20201013_0005_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c01_20201013_0009_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c01_20201013_0011_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c01_20201013_0012_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c01_20201013_0023_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c01_20201013_0025_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0003_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0005_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0007_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0012_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0014_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0030_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0031_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0032_S01.JPG을(를) 찾을 수 없습니다.
V006_77_0_00_11_03_12_0_c02_20201013_0033_S01_1.JPG을(를) 찾을 수 없습니다.
V006_77_0_0

In [9]:
import os
import shutil

# 경로 설정
data_dir = "data/tomato_yolo_shuffle_2"  # 데이터셋이 위치한 폴더 경로
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "test")
# val_dir = os.path.join(data_dir, "val")

# 이미지 파일과 텍스트 파일이 있는 폴더 경로
train_label_dir = os.path.join(train_dir, "labels")
val_image_dir = os.path.join(val_dir, "images")
val_label_dir = os.path.join(val_dir, "labels")

# val/labels 폴더가 없으면 생성
if not os.path.exists(val_label_dir):
    os.makedirs(val_label_dir)

# train/labels 폴더 내의 파일들과 val/images 폴더 내의 파일들 비교
train_labels = [os.path.splitext(f)[0] for f in os.listdir(train_label_dir)]
val_images = [os.path.splitext(f)[0] for f in os.listdir(val_image_dir)]

# val/images 폴더 내의 파일들 중 train/labels 폴더에 같은 이름이 있는 파일을 이동
for image_name in val_images:
    if image_name in train_labels:
        label_file = f"{image_name}.txt"
        label_path = os.path.join(train_label_dir, label_file)
        shutil.move(label_path, val_label_dir)

print("작업이 완료되었습니다.")


작업이 완료되었습니다.


In [12]:
from ultralytics import YOLO

model = YOLO("data/tomato_yolo_shuffle/best(shuffle_Epochs100).pt")

model.export(format="onnx")

Ultralytics YOLOv8.0.117  Python-3.9.13 torch-2.0.1+cpu CPU
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from data\tomato_yolo_shuffle\best(shuffle_Epochs100).pt with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 7, 3549) (5.9 MB)

ONNX: starting export with onnx 1.14.0 opset 17...
ONNX: export success  2.0s, saved as data\tomato_yolo_shuffle\best(shuffle_Epochs100).onnx (11.6 MB)

Export complete (2.4s)
Results saved to C:\juna27\PythonWork\AI\data\tomato_yolo_shuffle
Predict:         yolo predict task=detect model=data\tomato_yolo_shuffle\best(shuffle_Epochs100).onnx imgsz=416 
Validate:        yolo val task=detect model=data\tomato_yolo_shuffle\best(shuffle_Epochs100).onnx imgsz=416 data=/content/tomato_yolo_shuffle/tom_leaf_data.yaml 
Visualize:       https://netron.app


============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'data\\tomato_yolo_shuffle\\best(shuffle_Epochs100).onnx'